In [1]:
import numpy as np

In [2]:
x_size, y_size = 5, 11
x_arr = np.arange(x_size)
y_arr = np.arange(y_size)
grid = np.zeros((x_size, y_size,), dtype=np.float64)
x_grid = grid.copy()
y_grid = grid.copy()
x_grid[x_arr, :] = x_arr[:, None]
y_grid[:, y_arr] = y_arr

In [3]:
in_set = y_grid == 0
out_set = y_grid == (y_size - 1)
display(in_set, out_set)

array([[ True, False, False, False, False, False, False, False, False,
        False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False]])

array([[False, False, False, False, False, False, False, False, False,
        False,  True],
       [False, False, False, False, False, False, False, False, False,
        False,  True],
       [False, False, False, False, False, False, False, False, False,
        False,  True],
       [False, False, False, False, False, False, False, False, False,
        False,  True],
       [False, False, False, False, False, False, False, False, False,
        False,  True]])

In [4]:
resistor_grid = np.zeros((x_size, y_size, 2,), dtype=np.float64)
resistor_grid[:] = 1
display(resistor_grid[:, :, 0], resistor_grid[:, :, 1])

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [5]:
resistor_grid[:2, 4:7] = 0

In [6]:
display(resistor_grid[:, :, 0], resistor_grid[:, :, 1])

array([[1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

array([[1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [7]:
def calc_resistance(resistor_grid, in_set, out_set, *, total_iter=101, print_every_iter=10, dt=1/4):
    """
    resistor_grid.shape == (x_size, y_size, 2,)
    current_grid[x, y, 0] == current from (x,y) to (x+1,y)
    current_grid[x, y, 1] == current from (x,y) to (x,y+1)
    voltage[in_set] = 1.0
    voltage[out_set] = 0.0
    current_grid[x, y, 0] = (voltage[x, y] - voltage[x + 1, y]) * resistor_grid[x, y, 0]
    current_grid[x, y, 1] = (voltage[x, y] - voltage[x, y + 1]) * resistor_grid[x, y, 1]
    #
    Only need resistor_grid[:-1, :, 0] and resistor_grid[:, :-1, 1]
    #
    R = rho * L / S
    resistor_grid = 1 / rho # value of resistor_grid is actually conductance.
    """
    x_size, y_size, n_dim = resistor_grid.shape
    voltage = np.zeros((x_size, y_size,), dtype=np.float64)
    voltage[:] = 0.5
    voltage[in_set] = 1.0
    voltage[out_set] = 0.0
    for i in range(total_iter):
        current_grid = np.zeros((x_size, y_size, 2,), dtype=np.float64)
        current_grid[:-1, :, 0] = (voltage[:-1, :] - voltage[1:, :]) * resistor_grid[:-1, :, 0]
        current_grid[:, :-1, 1] = (voltage[:, :-1] - voltage[:, 1:]) * resistor_grid[:, :-1, 1]
        charge_change = np.zeros((x_size, y_size,), dtype=np.float64)
        charge_change[:-1, :] -= dt * current_grid[:-1, :, 0]
        charge_change[1:, :] += dt * current_grid[:-1, :, 0]
        charge_change[:, :-1] -= dt * current_grid[:, :-1, 1]
        charge_change[:, 1:] += dt * current_grid[:, :-1, 1]
        total_current_in = np.sum(charge_change[in_set]) / dt
        total_current_out = np.sum(charge_change[out_set]) / dt
        resistance_from_in = -1.0 / total_current_in
        resistance_from_out = 1.0 / total_current_out
        charge_change[in_set] = 0.0
        charge_change[out_set] = 0.0
        voltage += charge_change
        if i % print_every_iter == 0:
            print(i, np.linalg.norm(charge_change), resistance_from_in, resistance_from_out)
            # display(voltage)
    resistance = (resistance_from_in + resistance_from_out) / 2
    return resistance

In [8]:
calc_resistance(resistor_grid, in_set, out_set)

0 0.39528470752104744 0.4 0.4
10 0.06154616263877693 1.1894118821404913 1.1989137953704816
20 0.03092626110581347 1.6449793709472385 1.744136965206371
30 0.01647444917012824 1.9959468316226796 2.1892711899093125
40 0.009026049179777803 2.254189244416654 2.4867794716864244
50 0.00505238619214909 2.4322911693871148 2.6571813297983633
60 0.0028990060402877007 2.5501488539795 2.7446852283382954
70 0.0017209839954941974 2.626740447333319 2.785379836419328
80 0.0010684834283691276 2.6764647831461845 2.801760021134167
90 0.0006980654252486939 2.7090736886449083 2.8062980662896466
100 0.00047853994401356225 2.730811117250959 2.8055026556059546


2.7681568864284567